<a href="https://colab.research.google.com/github/sag30/summarization/blob/main/Summarization_T5_GPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Libraries

In [ ]:
pip install transformers

In [ ]:
from transformers import pipeline
import os

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

# Load Data

In [ ]:
import pandas as pd
df = pd.read_excel('/content/SummaryKeyWords2.xlsx')
df = df.head(30)
df.shape

In [ ]:
df.head(2)

# Summarization using default Transformer Pipline Summary model


In [ ]:
# summarizer = pipeline("summarization")
summarizer = pipeline("summarization", model="t5-base", tokenizer="t5-base", framework="tf")

In [ ]:
def summar(text):
  print("orig: ",text)
  summary_text = summarizer(text, max_length=80, min_length=50, do_sample=False)[0]['summary_text']
  print("summary: ",summary_text)
  return(summary_text)

In [ ]:
df = df.head(5)

In [ ]:
df['T5_Summary_80'] = df.Conclusion.apply(lambda x:summar(x))

# Summarization using T5-base Transformer

In [ ]:
!pip3 install sentencepiece

In [ ]:
from transformers import T5ForConditionalGeneration, AutoTokenizer

In [ ]:
# initialize the model architecture and weights
model = T5ForConditionalGeneration.from_pretrained("t5-base")

In [ ]:
# initialize the model tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-base")

In [ ]:
def summar_2(text):
  inputs = tokenizer.encode("summarize: " + text, return_tensors="pt", max_length=512, truncation=True)
  # generate the summarization output
  outputs = model.generate(
      inputs, 
      max_length=80, 
      min_length=50, 
      length_penalty=2.0, 
      num_beams=4, 
      early_stopping=True)
  # just for debugging
  # print(outputs)
  # print(tokenizer.decode(outputs[0]))
  return tokenizer.decode(outputs[0])

In [ ]:
text = "thank you for chatting frontier. this is loue. who am i speaking with please? hi amanda good morning! i hope you and your family are safe especially this time. thank you for bringing this to our attentionreaching us about this matter. a sudden increase on my bill would definitely concern me too. it’s great to know that you are fine and safe. i am doing fine too. lets go ahead and check whats different. you are talking to the best representative and i will make sure that this will be your last time chatting with a rep regarding this matter. i am here to help. before we begin just in case we get delays of our chat know that im here working for your concern. just please stay on the same page to ensure that we wont get disconnected. to get started allow me to pull up your account. can i have the billing telephone number associated with the account? for security purposes can you verify the 5digit passcode. thats alright. here is an easier step to locate your 5 digit passcode 1. please log in to your online account 2. go to &#34my profile&#34 tab and 3. from under that tab you can see &#34show passcode&#34 or get a new passcode. here is a much more convenient way to find your 5digit account passcode you can go to httpsfrontier.comhelpcentercategoriesaccountmanagemyaccountcreatechangeaccountpasscode then login to your account and follow the instruction on that page how you can create or change your passcode. i now have the account pulled up. let me check the account for a minute or two hold on. thank you for patiently waiting. based on the records the only thing thats caused an increase in your monthly rate charge for your fios internet is the expiration of its promotional price. the $10 savings you enjoyed from the time you availed that promotion is valid until 22022. i really wanted to give you an extended promotions for you to keep your amount what i can do is to process you a one time courtesy credit of $10 for your current bill as an appreciation for your being good credit standing customer. and to extend my help i will take $3 of your bill for three months but i know how this affecting your experience them i will make it for 6 months. you re awesome! aside from this are there any other concerns that needs to be addressed? please know that frontier cares for you and that is why i am here. thats how frontier cares. ) hold on please while ill make sure to apply the $10 off from your current bill and the $3 monthly credit for you then. hold on please. thank you for patiently waiting. to recap youve chatted about bill increase. as a resolution the $11.32 adjustment request has been approved and will be posted immediately to your account. and to extend my help i will take $3 of your bill for three months but i know how this affecting your experience them i will make it for 6 months starting next billing cycle. the order number is 062426676. i really enjoyed our conversation and this is what i needed to get me through today. all thanks to you. i thank you again for everything today. have i resolved the reason of our chat today amanda? you are most welcome. before you go if you don’t mind me asking how’s your experience with frontier so far? yes you are right. no worries i will leave comprehensive notes on your account so that if it becomes available any time soon you’ll be notified right away. will that be fine? thanks for the wonderful feedback! i hope you will spare a few minutes to answer a quick survey right after this chat should you want to leave a feedback on how i assisted you today. i will truly appreciate it. when you exit this chat kindly click the 3bar icon and choose end."
# text_summary_300 = "based on the records the only thing thats caused an increase in your monthly rate charge for your fios internet is the expiration of its promotional price. i really wanted to give you an extended promotions for you to keep your amount what i can do is to process you a one time courtesy credit of $10 for your current bill as an appreciation for your being good credit standing customer. i will take $3 of your bill for three months but i know how this affecting your experience them i will make it for 6"

# text = '["['you are most welcome!', 'is there anything else that i can be of help?', 'same to you.']"]'
# text = df['recap or last 3 sentences'][13]
print(text)
summar_2(text)

In [ ]:
def clean(text):
  text_new = text.replace("['",'').replace("]'",'').replace(".', '",". ").replace("', '"," ").replace('["',"")
  return text_new

df['clean_recap or last 3 sentences']=df['recap or last 3 sentences'].apply(lambda x : clean(x))
# df=df.reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
df['T5_Summary_80'] = df['T5_Summary_300'].apply(lambda x:summar_2(x))

In [ ]:
df.T5_Summary_80[0]

In [ ]:
for i in range(len(df.T5_Summary_80)):
  str_col = df.T5_Summary_80[i]
  str_col = str_col.replace("<pad>","")
  str_col = str_col.replace("</s>","")
  str_col = str_col.replace("'",'"')
  str_col = str_col.replace('" "',' ')
  str_col = str_col.replace('<unk>',"")
  str_col = str_col.replace(' "','"').replace('.","' ,". "). replace('","'," ")
  df.T5_Summary_80[i]=str_col

In [ ]:
df.T5_Summary_80[13]

In [ ]:
df.to_excel("/content/SummaryKeyWords4.xlsx")

In [ ]:
df.shape
df.columns

# Summarization using GPT3

In [ ]:
!pip install openai

In [ ]:
import openai
import pandas as pd
openai.api_key = "" ##Enter your open ai key here
ENGINE = "text-davinci-002"

#Using default params of 'Summarize for a 2nd grader' gpt3 preset
maxTokens = 64
TEMPERATURE = 0.7
TOP_P = 1
FREQ_PENALTY = 0
PRESENCE_PENALTY = 0


In [ ]:
def get_summary_gpt3(context):
    prompt = f"""Summarize this for a second-grade student:

    {context}"""
    prompt = prompt.strip()
    print(prompt)
    res = openai.Completion.create(
        engine=ENGINE,
        prompt=prompt,
        temperature=TEMPERATURE,
        max_tokens=maxTokens,
        top_p=TOP_P,
        frequency_penalty=FREQ_PENALTY,
        presence_penalty=PRESENCE_PENALTY
        # stop=['"""', "\\n\\n", "\\n", "\n"],
    )
    print(res["choices"][0]["text"])
    return res["choices"][0]["text"]

In [ ]:
df = pd.read_excel('/content/SummaryKeywordsSquadNew.xlsx')

In [ ]:
df['GPT3_Summary'] = df['recap or last 3 sentences'].apply(lambda x:get_summary_gpt3(x))

In [ ]:
df.to_excel("/content/SummaryKeywordSquadGPT3.xlsx")